In [269]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
import geopandas as gpd
from datetime import datetime, timedelta
import contextily as ctx

In [270]:
import matplotlib.animation as animation


In [271]:
# Columns to be used in the DataFrame
columns = ['icao24', 'callsign', 'origin_country', 'time_position', 'last_contact', 
           'longitude', 'latitude', 'baro_altitude', 'on_ground', 'velocity', 
           'true_track', 'vertical_rate', 'sensors', 'geo_altitude', 'squawk', 
           'spi', 'position_source']

In [272]:
# Microsoft outage 기간 동안의 시간 설정
start_time = datetime(2024, 7, 19, 0, 0, 0) #(2024, 7, 19, 10, 0, 0)
end_time = datetime(2024, 7, 19, 23, 59, 59) #(2024, 7, 19, 14, 0, 0)

# # 시간 범위를 Unix 타임스탬프로 변환
# start_timestamp = int(start_time.timestamp())
# end_timestamp = int(end_time.timestamp())

In [273]:
# 데이터 가져오기 함수
def fetch_flight_data(start_timestamp, end_timestamp):
    url = 'https://opensky-network.org/api/states/all'
    params = {
        'begin': start_timestamp,
        'end': end_timestamp,
        'lamin': 49.9,
        'lamax': 61.0,
        'lomin': -10.5,
        'lomax': 1.8
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"{start_timestamp}에서 {end_timestamp}까지의 기간에 대한 요청 실패: {e}")
        return None



In [274]:
# geo_df = fetch_flight_data()
# geo_df.to_csv('flightdata.csv')
# geo_df

In [275]:
# # Plot the aircraft positions
# plt.figure(figsize=(10, 6))
# geo_df.plot(marker='o', color='red', markersize=5, alpha=0.5)
# plt.title('Aircraft Positions on July 19, 2024')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.show()

In [ ]:
# 모든 비행 데이터를 저장할 빈 DataFrame 초기화
all_flight_data = pd.DataFrame()

# 각 시간마다 데이터를 가져와 DataFrame에 추가
current_time = start_time
while current_time < end_time:
    next_time = current_time + timedelta(hours=1) #(minutes=30) (hours=1)
    start_timestamp = int(current_time.timestamp())
    end_timestamp = int(next_time.timestamp())
    
    for attempt in range(5):  # 5번의 시도 재시도 로직
        data = fetch_flight_data(start_timestamp, end_timestamp)
        if data:
            if 'states' in data and data['states']:
                # 반환된 데이터의 열 이름을 확인하고 동적으로 처리
                columns = ["icao24", "callsign", "origin_country", "time_position", "last_contact", 
                           "longitude", "latitude", "baro_altitude", "on_ground", "velocity", 
                           "true_track", "vertical_rate", "sensors", "geo_altitude", "squawk", 
                           "spi", "position_source"]
                if len(data['states'][0]) == len(columns):
                    df = pd.DataFrame(data['states'], columns=columns)
                    df['timestamp'] = current_time
                    all_flight_data = pd.concat([all_flight_data, df], ignore_index=True)
                    break
                else:
                    print(f"{current_time}부터 시작하는 기간에 대한 데이터 구조가 예상과 다릅니다.")
                    print(f"예상된 열 수: {len(columns)}, 반환된 열 수: {len(data['states'][0])}")
                    print(f"반환된 데이터: {data['states'][0]}")
                    break
            else:
                print(f"{current_time}부터 시작하는 기간에 대한 데이터 없음")
                break
        else:
            print(f"{start_timestamp}에서 {end_timestamp}까지의 기간에 대한 {attempt + 1}/5 재시도 중")
            time.sleep(10 * (attempt + 1))  # 재시도 간 대기 시간 증가
    
    current_time = next_time
    time.sleep(1)  # 각 요청 사이에 1초 대기

# 결합된 DataFrame의 처음 몇 줄 표시
print(all_flight_data.head())

In [ ]:
all_flight_data.to_csv('uk_flights_july_19_2024_30min.csv', index=False)

In [ ]:
all_flight_data

In [ ]:
# 항공기 분류 -> icao24 (총 318개 항공기)